In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [2]:
train=pd.read_csv('Final_Train_Dataset.csv')
test=pd.read_csv('Final_Test_Dataset.csv')
train.head()

,Unnamed: 0,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
0,0,5-7 yrs,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,NaN,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,1,10-17 yrs,He should have handled a team of atleast 5-6 d...,Head SCM,NaN,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458
2,2,5-9 yrs,Must be an effective communicator (written & s...,Deputy Manager - Talent Management & Leadershi...,Analytics,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195
3,3,7-10 yrs,7 - 10 years of overall experience in data e...,Associate Manager Data Engineering,Analytics,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313
4,4,1-3 yrs,Chartered Accountancy degree or MBA in Finance...,TS- GSA- Senior Analyst,NaN,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305


In [3]:
train.company_name_encoded.nunique()
train.shape

(19802, 9)

In [4]:
train.salary.value_counts()
print(1)
print(2)

1
2


In [5]:
test.head()

,experience,job_description,job_desig,job_type,key_skills,location,company_name_encoded
0,7-12 yrs,Professional experience in Java/J2EE based ser...,IT Technology Senior Consultant/java/ J2ee/ Se...,NaN,"Java, J2Ee, Tomcat, JBoss, Weblogic, Oracle, E...",Bengaluru,2066
1,0-5 yrs,We are looking for 20+ Fresher/Experienced Can...,Medical Billing Process | International KPO | ...,NaN,"Medical Billing, Insurance Processing",Ahmedabad(Sola),2629
2,3-6 yrs,Should understand overall integration framewor...,Oracle Sales Cloud Functional Consultant,NaN,"Oracle Sales, Functional Consultancy, Troubles...",Bengaluru,2448
3,0-3 yrs,NaN,Looking For Freshers WHO WANT To Work WITH US,NaN,"offline, online, internet, part time, home bas...","Delhi NCR, Chennai, Hyderabad, Gurgaon, Luckno...",2711
4,0-5 yrs,NaN,Process Associate / Sr Process Associate / Tec...,NaN,"voice support, analytical skills, Process asso...",Hyderabad,40


In [6]:
train.job_type.value_counts()

Analytics    3747
analytics     921
ANALYTICS      77
analytic       32
Analytic       20
Name: job_type, dtype: int64

In [7]:
df=train.copy()

df.job_description.fillna('None',inplace=True)
df.key_skills.fillna('None',inplace=True)
def exp1(val):
    v=val.split(" ")[0].split("-")
    v1,v2=int(v[0]),int(v[1])
    return (v1+v2)/2
df['exp']=df.experience.apply(exp1)
df['key_skill_len']=df.key_skills.apply(lambda x:len(x.split(",")))


df['exp_low']=df.experience.apply(lambda x:int(x.split("-")[0]))
df['exp_high']=df.experience.apply(lambda x:int(x.split("-")[1].split(" ")[0]))


df['jdescLen']=df.job_description.apply(len)
df['jdsgnLen']=df.job_desig.apply(len)
df['locationLen']=df.location.apply(lambda x:len(x.split(",")))

df.drop(['job_type','Unnamed: 0','experience'],axis=1,inplace=True)
target=df.salary

In [8]:
df.head()

,job_description,job_desig,key_skills,location,salary,company_name_encoded,exp,key_skill_len,exp_low,exp_high,jdescLen,jdsgnLen,locationLen
0,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687,6.0,4,5,7,96,43,1
1,He should have handled a team of atleast 5-6 d...,Head SCM,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458,13.5,6,10,17,108,8,1
2,Must be an effective communicator (written & s...,Deputy Manager - Talent Management & Leadershi...,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195,7.0,4,5,9,100,66,1
3,7 - 10 years of overall experience in data e...,Associate Manager Data Engineering,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313,8.5,7,7,10,106,34,1
4,Chartered Accountancy degree or MBA in Finance...,TS- GSA- Senior Analyst,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305,2.0,5,1,3,107,23,1


In [9]:
df.isnull().sum()

job_description         0
job_desig               0
key_skills              0
location                0
salary                  0
company_name_encoded    0
exp                     0
key_skill_len           0
exp_low                 0
exp_high                0
jdescLen                0
jdsgnLen                0
locationLen             0
dtype: int64

In [10]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv_jb = TfidfVectorizer(ngram_range=(1,3),stop_words="english", analyzer='word')
jb =cv_jb.fit_transform(df['job_description'])

cv_jd = TfidfVectorizer(ngram_range=(1,2),stop_words="english", analyzer='word')
jd =cv_jd.fit_transform(df['job_desig'])

cv_key = TfidfVectorizer(ngram_range=(1,3),stop_words="english", analyzer='word')
ks =cv_key.fit_transform(df['key_skills'])

cv_jbchar = TfidfVectorizer(ngram_range=(1,9),stop_words="english", analyzer='char')
jbchar =cv_jbchar.fit_transform(df['job_description'])

cv_jdchar = TfidfVectorizer(ngram_range=(1,8),stop_words="english", analyzer='char')
jdchar =cv_jdchar.fit_transform(df['job_desig'])

cv_keychar = TfidfVectorizer(ngram_range=(1,9),stop_words="english", analyzer='char')
kschar =cv_keychar.fit_transform(df['key_skills'])

In [11]:
loc = np.append(df['location'].values, test['location'].values, axis=0)
loc

array(['Delhi NCR(Vikas Puri)', 'Sonepat', 'Delhi NCR', ..., 'Mumbai',
       'Hyderabad', 'Bengaluru'], dtype=object)

In [12]:
loc.shape

(26403,)

In [13]:
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
df['location']=l.transform(df['location'])


tot_comp = np.append(df['company_name_encoded'].values, test['company_name_encoded'].values, axis=0)
l_comp=LabelEncoder()
l_comp.fit(list(set(tot_comp)))
df['company_name_encoded']=l_comp.transform(df['company_name_encoded'])

In [14]:
df.columns

Index(['job_description', 'job_desig', 'key_skills', 'location', 'salary',
       'company_name_encoded', 'exp', 'key_skill_len', 'exp_low', 'exp_high',
       'jdescLen', 'jdsgnLen', 'locationLen'],
      dtype='object')

In [15]:
from scipy.sparse import csr_matrix
from scipy import sparse
final_features = sparse.hstack((df[['location','company_name_encoded','exp','key_skill_len','jdescLen', 'jdsgnLen', 'locationLen']], jb, jd, ks,jbchar, jdchar, kschar)).tocsr()

In [16]:
final_features

<19802x2102744 sparse matrix of type '<class 'numpy.float64'>'
	with 26948727 stored elements in Compressed Sparse Row format>

In [17]:
dftest=test.copy()

dftest.job_description.fillna('None',inplace=True)
dftest.key_skills.fillna('None',inplace=True)
def exp1(val):
    v=val.split(" ")[0].split("-")
    v1,v2=int(v[0]),int(v[1])
    return (v1+v2)/2
dftest['exp']=dftest.experience.apply(exp1)
dftest['key_skill_len']=dftest.key_skills.apply(lambda x:len(x.split(",")))

dftest['exp_low']=dftest.experience.apply(lambda x:int(x.split("-")[0]))
dftest['exp_high']=dftest.experience.apply(lambda x:int(x.split("-")[1].split(" ")[0]))

jb_t =cv_jb.transform(dftest['job_description'])
jd_t =cv_jd.transform(dftest['job_desig'])
ks_t =cv_key.transform(dftest['key_skills'])

jb_tchar =cv_jbchar.transform(dftest['job_description'])
jd_tchar =cv_jdchar.transform(dftest['job_desig'])
ks_tchar =cv_keychar.transform(dftest['key_skills'])


dftest['location']=l.transform(dftest['location'])
dftest['company_name_encoded']=l_comp.transform(dftest['company_name_encoded'])

dftest['jdescLen']=dftest.job_description.apply(len)
dftest['jdsgnLen']=dftest.job_desig.apply(len)
dftest['locationLen']=dftest.location.astype(str).apply(lambda x:len(x.split(",")))
dftest.drop(['job_type','experience'],axis=1,inplace=True)


final_features_t = sparse.hstack((dftest[['location','company_name_encoded','exp','key_skill_len','jdescLen', 'jdsgnLen', 'locationLen']], jb_t, jd_t, ks_t,jb_tchar, jd_tchar, ks_tchar)).tocsr()

In [21]:
final_features_t

<6601x2102744 sparse matrix of type '<class 'numpy.float64'>'
	with 8669216 stored elements in Compressed Sparse Row format>

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train,X_val,y_train,y_val = train_test_split(final_features,target,test_size=0.25,random_state = 1994)

In [38]:
X = train.drop(columns=['key_skills', 'salary'], axis=1)
y = train['salary']

In [39]:
from sklearn.model_selection import train_test_split, cross_val_score
def train_model(model):
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    model.fit(x_train, y_train)
    print('Accuracy:', model.score(x_test, y_test))
    
    score = cross_val_score(model, X, y, cv=5)
    print('CV Score:', np.mean(score))

In [40]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
train_model(model)

ValueError: could not convert string to float: 'Bengaluru'